<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Chat Completion

Decision Intelligence applied in this module:  
* Listing of various decision-making frameworks and with their descriptions by using Chat Completion  
* Decision Scenario: Recommendation of a decision-making framework for specific higher-stakes decisions (purchase of a car)  
* Illustrating a domain specific decision-making framework (millitary intelligence)  


In this module, the Semantic Kernel ability to create a chat experience will be introduced. This is a much richer experience than just sending simple prompts that are stateless and context is forgotten.

Semantic Kernel has first-class support for chat scenarios, where the user talks back and forth with the LLM, the arguments get populated with the history of the conversation. During each new run of the kernel, the arguments will be provided to the AI with content. This allows the LLM to know the historical context of the conversation.

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [1]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"

using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0

In [2]:
// Install the required NuGet packages
// Note: This also installs the Dependency Injection Package to retrieve the ChatCompletionService
#r "nuget: Microsoft.Extensions.DependencyInjection, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.20.0"

using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

var chatCompletionService = semanticKernel.Services.GetRequiredService<IChatCompletionService>();

Installed Packages Microsoft.Extensions.DependencyInjection, 8.0.0 Microsoft.SemanticKernel, 1.20.0

Using Azure OpenAI Service


### Step 2 - Execute Chat Completion with Prompt Execution Settings

> "If you can’t describe what you are doing as a process, you don’t know what you’re doing." 
>
> -- <cite>W. Edwards Deming (renowned American engineer, statistician, and management consultant)</cite>  


Using the Semantic Kernel ChatCompletion service is very similar to inovoking a prompt for basic LLM interactions. The chat completion service will provide very similar results to invoking the prompt directly.

In [3]:
// Simple prompt to list some decision frameworks this GenAI LLM is familiar with 
// LLMs are trained on a diverse range of data and can provide insights on a wide range of topics like decision frameworks
// SLMS (smaller LLMs) are trained on a more specific range of data and may not provide insights on all topics
var simpleDecisionPrompt = """
Provide a list of decision frameworks that can help improve the quality of decisions.
""";

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 2500, 
    Temperature = 0.1, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

var chatResult = await chatCompletionService.GetChatMessageContentAsync(simpleDecisionPrompt, openAIPromptExecutionSettings);

Console.WriteLine(chatResult.Content);

Certainly! Here’s a list of decision frameworks that can help improve the quality of decisions:

1. **SWOT Analysis**:
   - Evaluates Strengths, Weaknesses, Opportunities, and Threats related to a decision or project.

2. **Decision Matrix**:
   - A grid that helps prioritize options based on specific criteria, allowing for a systematic comparison.

3. **Cost-Benefit Analysis**:
   - Compares the costs and benefits of different options to determine the most economically viable choice.

4. **Pareto Analysis**:
   - Focuses on identifying the most significant factors contributing to a problem, based on the 80/20 rule.

5. **Multi-Criteria Decision Analysis (MCDA)**:
   - Evaluates multiple conflicting criteria in decision-making, often using weighted scoring.

6. **The Eisenhower Matrix**:
   - Prioritizes tasks based on urgency and importance, helping to focus on what truly matters.

7. **OODA Loop (Observe, Orient, Decide, Act)**:
   - A decision-making process that emphasizes agility 

### Step 3 - Execute a Semantic Kernel Chat Completion with Streaming

Semantic Kernel supports prompt response streaming in the ChatCompletionService. This allows responses to be streamed to the client as soon as they are made available by the LLM and API services. Below the same decision prompt is executed. However, notice that chunks are streamed and can be read by the user as soon as they appear. 

In [4]:
// Stream the chat completion response
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(simpleDecisionPrompt))
{
    Console.Write(content);
}

Here are several decision frameworks that can enhance the quality of decision-making:

1. **SWOT Analysis**: Evaluates Strengths, Weaknesses, Opportunities, and Threats related to a decision or strategic initiative.

2. **DECIDE Model**: A structured approach including Define the problem, Establish criteria, Consider alternatives, Identify the best alternative, Develop and implement a plan, and Evaluate the decision.

3. **Cost-Benefit Analysis (CBA)**: Compares the costs and benefits of different options to determine the most economically advantageous choice.

4. **Decision Matrix (Pugh Matrix)**: A grid that evaluates and prioritizes options based on multiple criteria, helping to visually represent trade-offs.

5. **Pareto Analysis**: Focuses on identifying the most impactful factors in a decision, often summarized by the 80/20 rule, which suggests that 80% of effects come from 20% of the causes.

6. **Risk Assessment Framework**: Analyzes potential risks associated with different de

### Step 4 - Execute a Semantic Kernel with Chat History

Semantic Kernel has a ChatHistory object that can be used with the ChatCompletionService to provide historical chat context to the LLM. Notice that the ChatHistory object differentiates between the different types of chat messages:
* System Message - System or MetaPrompt. These are usually global instructions that set the "overall rules" for interacting with the LLMs.
* User Message - A message from the user
* Assistant Message - A message from the LLM. This is a message generated from an assistant or an agent. 

Identifying the messages from which role (user) it came from can help the LLM improve its own reasoning responses. This is a more sophisticated approach than passing chat history in a long dynamic string. ChatHistory objects can be serialized and persisted into databases as well.

In [6]:
// Set the overall system prompt to behave like a decision intelligence assistant (persona)
var systemPrompt = """
You are a decision intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";

// Simple instruction prompt to list 5 (five) decision frameworks this GenAI LLM is familiar with
var simpleDecisionPrompt = """
Provide five decision frameworks that can help improve the quality of decisions.
""";

var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPrompt);
chatHistory.AddUserMessage(simpleDecisionPrompt);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}
chatHistory.AddAssistantMessage(chatResponse);

Certainly! Here are five decision frameworks that can enhance decision-making quality:

### 1. **SWOT Analysis**
   - **Description**: SWOT (Strengths, Weaknesses, Opportunities, Threats) analysis helps evaluate internal and external factors influencing a decision.
   - **How to Use**:
     - **Strengths**: Identify what advantages you have.
     - **Weaknesses**: Recognize limitations or areas that need improvement.
     - **Opportunities**: Explore external chances to grow or improve.
     - **Threats**: Acknowledge external challenges that could hinder progress.
   - **Application**: Useful for strategic planning and evaluating business initiatives.

### 2. **Decision Matrix Analysis**
   - **Description**: This framework helps in evaluating and prioritizing multiple options based on various criteria.
   - **Steps**:
     1. List options on one axis and criteria on the other.
     2. Rate each option against the criteria.
     3. Assign weights to each criterion based on importance.

### Step 5 - Using Semantic Kernel Chat History in Conversation

With Semantic Kernel maintaining state with the ChatHistory object, a user can interact with the LLM in a natural way. Note the interaction below, where the user is asking which of the particular decision frameworks are best suited to purchase a car. Without the ChatHistory being passed along, the LLM would not have had the proper context to answer the question.

> "Price is what you pay. Value is what you get."  
>
> -- <cite>Warren Buffet (one of the most successful investors of all time)</cite>  

In [7]:
// Simple prompt that asks a follow-up question about the purchase of a car
// Note: No reference is made to what previous frameworks were listed.
// Note: Previous context is maintained by the Semantic Kernel ChatHistory object 
var simpleDecisionPromptFollowup = """
Which of these 5 decision frameworks would work best for making a decision to purchase a car. 
Select only the single best one.
""";

// Add User message to the chat history
chatHistory.AddUserMessage(simpleDecisionPromptFollowup);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}

// Add Assistant message to the chat history
chatHistory.AddAssistantMessage(chatResponse);

For making a decision to purchase a car, the **Decision Matrix Analysis** would be the best choice. 

### Rationale:
1. **Multiple Options**: When purchasing a car, there are often several models, brands, and types (e.g., sedan, SUV, electric, hybrid) to consider.
2. **Criteria for Evaluation**: Different criteria can be weighed, such as price, fuel efficiency, safety ratings, maintenance costs, and features (e.g., technology, comfort).
3. **Quantitative Approach**: By assigning scores and weights to criteria, you can create a structured approach to objectively evaluate and compare your options, helping to alleviate decision fatigue.

### Steps to Implement:
1. **List Car Options**: Write down the various car models you are considering.
2. **Define Criteria**: Identify important criteria for your decision (e.g., cost, reliability, resale value, insurance rates).
3. **Rate Each Option**: Score each car based on how well it meets each criterion.
4. **Assign Weights**: Determine which cri

> "Take time to deliberate, but when the time for action comes, stop thinking and go in."
>
> -- <cite>Napoleon Bornaparte (French Emporer and brilliant military mind)</cite>  

In [8]:
// Note: No reference is made to what previous frameworks were listed.
// Note: Previous context is maintained by the Semantic Kernel ChatHistory object 
var simpleDecisionPromptFollowupPartTwo = """
Which of these 5 decision frameworks listed above are used by the military intelligence community?
""";

// Add User message to the chat history
chatHistory.AddUserMessage(simpleDecisionPromptFollowupPartTwo);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}

// Add Assistant message to the chat history
chatHistory.AddAssistantMessage(chatResponse);

The **OODA Loop (Observe, Orient, Decide, Act)** is predominantly used by the military intelligence community. 

### Rationale:
1. **Dynamic Environments**: The OODA Loop is designed for fast-paced and often chaotic situations where quick decision-making is crucial, such as in military operations and intelligence assessments.
2. **Iterative Process**: It emphasizes continuous observation and adaptation, allowing military leaders to respond to changing conditions on the battlefield or during intelligence operations.
3. **Situational Awareness**: The framework encourages a deep understanding of the operational environment, which is essential for effective strategic planning and decision-making in military contexts.

### Application in Military Intelligence:
- The OODA Loop helps military strategists analyze enemy actions and intentions, adjust tactics, and improve decision-making processes.
- It supports commanders in making timely and informed choices based on real-time intelligence and

### Step 6 - Semantic Kernel inspecting Chat History 

The Semantic Kernel Chat History is a transparent object that can be inspected and written out. In fact, the ChatHistory object can be manipulated to replay chats from middle interactions to simulate different outcomes.

Execute the cell below to write out the Semantic Kernel Chat History of this chat.

In [9]:
// Print the number of chat interactions and the chat history (turns)
Console.WriteLine("Number of chat interactions: " + chatHistory.Count());

foreach (var message in chatHistory)
{
    Console.WriteLine($"# {message.Role}: {message.Content}");
}

Number of chat interactions: 7
# system: You are a decision intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
# user: Provide five decision frameworks that can help improve the quality of decisions.
# assistant: Certainly! Here are five decision frameworks that can enhance decision-making quality:

### 1. **SWOT Analysis**
   - **Description**: SWOT (Strengths, Weaknesses, Opportunities, Threats) analysis helps evaluate internal and external factors influencing a decision.
   - **How to Use**:
     - **Strengths**: Identify what advantages you have.
     - **Weaknesses**: Recognize limitations or areas that need improvement.
     - **Opportunities**: Explore external chances to grow or improve.
     - **Threats**: Acknowledge external challenges that could hinder progress.
   - **Application**: Useful for strategic planni